[![kaggle](https://img.shields.io/badge/kaggle-CLICK%20ME!-%2320BEFF?style=for-the-badge&logo=kaggle)](https://www.kaggle.com/egorbronnikov/bridge-svm-knn)

<div align="center"><h2>Метод опорных векторов для классификации изображений</h2></div>

__С выбросами__

In [2]:
import os
import cv2
import numpy as np
import pandas as pd

from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [3]:
def get_img_data(type_: str):
    result = []
    directory = os.fsencode(f"../input/aleksandranevskogo/{type_}/")
    if type_ == "down":
        d_type_ = 0
    elif type_ == "up":
        d_type_ = 1
    elif type_ == "mov":
        d_type_ = 2

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".jpeg"):
            img = cv2.imread(f"../input/aleksandranevskogo/{type_}/{filename}")
            cropped_image = img[35:70, 420:480]
            cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
            cropped_image = cropped_image / 255.
            cropped_image = cv2.resize(cropped_image, (10, 10), interpolation=cv2.INTER_AREA)
            result.append(cropped_image.reshape(-1))
            result[-1] = np.append(result[-1], d_type_)
        else:
            continue
    return pd.DataFrame(result)

In [4]:
data_down = get_img_data("down")
data_up = get_img_data("up")
data_mov = get_img_data("mov")

In [5]:
df_final = pd.concat([data_down, data_up, data_mov], ignore_index=True)

In [6]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
svc = LinearSVC(random_state=42, dual=False)
knn = KNeighborsClassifier()

In [7]:
max_c = [0.01, 0.1, 1, 5, 10, 100]
params_svc = {'C': max_c}

gs_svc = GridSearchCV(svc,
                      param_grid=params_svc,
                      scoring='accuracy',
                      n_jobs=4,
                      cv=kf)

In [8]:
max_n_neighbors = [3, 5, 7, 9, 10, 12]
params_knn = {'n_neighbors': max_n_neighbors}

gs_knn = GridSearchCV(knn,
                      param_grid=params_knn,
                      scoring='accuracy',
                      n_jobs=4,
                      cv=kf)

In [9]:
X = df_final.loc[:, 0:99]
y = df_final.loc[:, 100]

In [10]:
gs_svc.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LinearSVC(dual=False, random_state=42), n_jobs=4,
             param_grid={'C': [0.01, 0.1, 1, 5, 10, 100]}, scoring='accuracy')

In [11]:
print(f"Best parameters: {gs_svc.best_params_}")
print(f"Accuracy score: {gs_svc.score(X, y)}")

Best parameters: {'C': 10}
Accuracy score: 0.8973565804274466


In [12]:
gs_knn.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': [3, 5, 7, 9, 10, 12]},
             scoring='accuracy')

In [13]:
print(f"Best parameters: {gs_knn.best_params_}")
print(f"Accuracy score: {gs_knn.score(X, y)}")

Best parameters: {'n_neighbors': 3}
Accuracy score: 0.9749718785151856


In [14]:
y_pred_svc = gs_svc.predict(X)
y_pred_knn = gs_knn.predict(X)

In [15]:
confusion_matrix(y, y_pred_svc)

array([[1451,    0,   17],
       [  35, 1053,   19],
       [ 140,  154,  687]])

In [16]:
confusion_matrix(y, y_pred_knn)

array([[1443,    3,   22],
       [   5, 1082,   20],
       [  17,   22,  942]])

__Избавление от выбросов__

Будем отфильтровывать выбросы следующим образом:
1. находим среднее значение красного, зелёного, синего в изображении;
1. если все эти среднии значения больше определённого показателя (допустим 30), это означает, что изображение чёрное и скорее всего это выброс;
1. аналогично поступаем с пересвеченными изображениям.

In [17]:
def mean(image, *, channels=3):
    result = [0 for _ in range(channels)]

    for i in range(channels):
        result[i] = np.mean(image[:, :, i])
    
    return np.array(result)

In [18]:
def get_img_filtered_data(type_: str):
    result = []
    directory = os.fsencode(f"../input/aleksandranevskogo/{type_}/")
    if type_ == "down":
        d_type_ = 0
    elif type_ == "up":
        d_type_ = 1
    elif type_ == "mov":
        d_type_ = 2

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".jpeg"):
            img = cv2.imread(f"../input/aleksandranevskogo/{type_}/{filename}")
            cropped_image = img[35:70, 420:480]
            # отфильтровываем выбросы
            if all(mean(cropped_image) < 40) or all(mean(cropped_image) > 220):
                continue
            cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
            cropped_image = cropped_image / 255.
            cropped_image = cv2.resize(cropped_image, (10, 10), interpolation=cv2.INTER_AREA)
            result.append(cropped_image.reshape(-1))
            result[-1] = np.append(result[-1], d_type_)
        else:
            continue
    return pd.DataFrame(result)

In [19]:
filtered_data_down = get_img_filtered_data("down")
filtered_data_up = get_img_filtered_data("up")
filtered_data_mov = get_img_filtered_data("mov")

In [20]:
df_filtered_final = pd.concat([filtered_data_down, filtered_data_up, filtered_data_mov], ignore_index=True)

In [21]:
print(f"Количество изображений в прошлый раз: {df_final.shape[0]}")
print(f"Количество отобранных изображений: {df_filtered_final.shape[0]}")

Количество изображений в прошлый раз: 3556
Количество отобранных изображений: 3191


In [22]:
X_filtered = df_filtered_final.loc[:, 0:99]
y_filtered = df_filtered_final.loc[:, 100]

In [23]:
gs_svc.fit(X_filtered, y_filtered)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LinearSVC(dual=False, random_state=42), n_jobs=4,
             param_grid={'C': [0.01, 0.1, 1, 5, 10, 100]}, scoring='accuracy')

In [24]:
print(f"Best parameters: {gs_svc.best_params_}")
print(f"Accuracy score: {gs_svc.score(X_filtered, y_filtered)}")

Best parameters: {'C': 10}
Accuracy score: 0.9066123472265747


In [25]:
gs_knn.fit(X_filtered, y_filtered)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': [3, 5, 7, 9, 10, 12]},
             scoring='accuracy')

In [26]:
print(f"Best parameters: {gs_knn.best_params_}")
print(f"Accuracy score: {gs_knn.score(X_filtered, y_filtered)}")

Best parameters: {'n_neighbors': 3}
Accuracy score: 0.9768097774992165


In [27]:
y_pred_filtered_svc = gs_svc.predict(X_filtered)
y_pred_filtered_knn = gs_knn.predict(X_filtered)

Матрица возмущений _до_ фильтрации выбросов (SVC):

In [28]:
confusion_matrix(y, y_pred_svc)

array([[1451,    0,   17],
       [  35, 1053,   19],
       [ 140,  154,  687]])

Матрица возмущений _после_ фильтрации выбросов (SVC):

In [29]:
confusion_matrix(y_filtered, y_pred_filtered_svc)

array([[1161,    0,   16],
       [   0, 1047,   14],
       [ 118,  150,  685]])

Можно заметить, что фильтрация выбросов положительно сказалась на предсказаниях и модель стала меньше ошибаться.

Матрица возмущений _до_ фильтрации выбросов (KNN):

In [30]:
confusion_matrix(y, y_pred_knn)

array([[1443,    3,   22],
       [   5, 1082,   20],
       [  17,   22,  942]])

Матрица возмущений _после_ фильтрации выбросов (KNN):

In [31]:
confusion_matrix(y_filtered, y_pred_filtered_knn)

array([[1155,    1,   21],
       [   2, 1040,   19],
       [   9,   22,  922]])

Можно заметить, что фильтрация выбросов положительно сказалась на предсказаниях и модель стала меньше ошибаться.